In [1]:
import cv2
import numpy as np
import scipy
import imageio
import glob, os
from skimage.registration import phase_cross_correlation
from plot_utils import *
from tqdm import tqdm

image_paths = glob.glob('retina2/*.bmp')
input_images = [cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) for image_path in image_paths]

In [2]:
def generate_gif(images, filename, shift=False):

    with imageio.get_writer(filename, mode='I', fps=30) as writer:
        
        if shift:
            first_image = images[0]
            for image in tqdm(images, desc='Applying shifts'):
                shift, error, diffphase = phase_cross_correlation(first_image, image, upsample_factor=100,
                                                                  overlap_ratio=0.9, normalization=None)
                writer.append_data(scipy.ndimage.shift(image, shift))
        else:
            for image in images:
                writer.append_data(image)

### Temporal Averaging

In [3]:
tavgN, label_offset = 20, 172
UNLOCK_LOOP = False  # Set to True to run the loop

output_dir = f'viz_outputs/tavg_{tavgN}'
os.makedirs(output_dir, exist_ok=True)
shifts = []

'''
1) For each image, we calculate the phase cross correlation with the previous tavgN images.
2) We then shift the previous tavgN images by the calculated shifts.
   The shift is calculate w.r.t. the last image to simulate averaging the image history.
3) We then average the shifted images to get the temporal average.
4) We then calculate the maximum and minimum shifts in the x and y directions.
5) We then save the temporal average image and the shifts.

Note: The shifts are saved as (shift_lr_pos, shift_lr_neg, shift_ud_pos, shift_ud_neg) which are used for masking later.
'''

assert UNLOCK_LOOP, 'Set UNLOCK_LOOP to True to run the loop'


for i in tqdm(range(tavgN-1, len(input_images)), desc='Temporal averaging'):

    last_image = input_images[i]
    corrected_images = []

    shift_lr_pos, shift_lr_neg = 0, 0
    shift_ud_pos, shift_ud_neg = 0, 0

    for j in range(tavgN):

        offset_image = input_images[i-j]
        shift, error, diffphase = phase_cross_correlation(last_image, offset_image, upsample_factor=100,
                                                          overlap_ratio=0.9, normalization=None)
        corrected_image = scipy.ndimage.shift(offset_image, shift)
        corrected_images.append(corrected_image)

        if shift[1] > 0:
            shift_lr_pos = max(shift_lr_pos, shift[1])
        else:
            shift_lr_neg =  min(shift_lr_neg, shift[1])

        if shift[0] > 0:
            shift_ud_pos = max(shift_ud_pos, shift[0])
        else:
            shift_ud_neg = min(shift_ud_neg, shift[0])
    
    temporal = np.mean(corrected_images, axis=0).astype(np.uint8)
    cv2.imwrite(f'{output_dir}/{i-tavgN+label_offset+1}-{i+label_offset}.bmp', temporal)

    shift_lr_pos = int(shift_lr_pos) + 1
    shift_lr_neg = int(shift_lr_neg) - 1
    shift_ud_pos = int(shift_ud_pos) + 1
    shift_ud_neg = int(shift_ud_neg) - 1

    shift_lr_pos = shift_lr_pos if shift_lr_pos > 0 else 0
    shift_lr_neg = shift_lr_neg if shift_lr_neg < 0 else -1
    shift_ud_pos = shift_ud_pos if shift_ud_pos > 0 else 0
    shift_ud_neg = shift_ud_neg if shift_ud_neg < 0 else -1

    shifts.append((shift_lr_pos, shift_lr_neg, shift_ud_pos, shift_ud_neg))

shifts = np.array(shifts)
np.save(f'{output_dir}/shifts.npy', shifts)

AssertionError: Set UNLOCK_LOOP to True to run the loop

In [4]:
tavg_image_paths = glob.glob(f'{output_dir}/*.bmp')
tavg_input_images = [cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) for image_path in tavg_image_paths]

np.load(f'{output_dir}/shifts.npy').shape

(210, 4)

### Wiener Deconvolution

Use MATLAB scripts to generate results `final_all_matlab.m`

In [5]:
w_avg_image_paths = glob.glob('viz_outputs/tavg_20_matlab_edges/*.mat')
w_avg_input_images = [scipy.io.loadmat(image_path)['wnr_avg'] for image_path in w_avg_image_paths]

w_dsk_image_paths = glob.glob('viz_outputs/tavg_20_matlab_edges/*.mat')
w_dsk_input_images = [scipy.io.loadmat(image_path)['wnr_dsk'] for image_path in w_dsk_image_paths]

### Generate GIF of images

<img src="viz_outputs/retina2_original.gif" alt="original"/><br>
<img src="viz_outputs/retina2_tvg_20_shifted.gif" alt="temporal averaging"/>
<img src="viz_outputs/retina2_wavg_20_shifted.gif" alt="wiener deconvolution: average"/>
<img src="viz_outputs/retina2_wdsk_20_shifted.gif" alt="wiener deconvolution: disk"/>

In [6]:
## Uncomment this block to generate a GIF of the images

# generate_gif(input_images, 'viz_outputs/retina2_original.gif')
# generate_gif(tavg_input_images, 'viz_outputs/retina2_tvg_20.gif')
# generate_gif(tavg_input_images, 'viz_outputs/retina2_tvg_20_shifted.gif', shift=True)

# generate_gif(w_avg_input_images, 'viz_outputs/retina2_wavg_20.gif')
# generate_gif(w_avg_input_images, 'viz_outputs/retina2_wavg_20_shifted.gif', shift=True)
# generate_gif(w_dsk_input_images, 'viz_outputs/retina2_wdsk_20.gif')
# generate_gif(w_dsk_input_images, 'viz_outputs/retina2_wdsk_20_shifted.gif', shift=True)